In [1]:
import torch
from torch import nn
from torch.optim import Adam
from torch.nn import MSELoss

from tqdm.notebook import tqdm

from preprocess import preprocess_and_return_loaders

In [2]:
assert torch.cuda.is_available()
device = 'cuda'

In [3]:
dataloader = preprocess_and_return_loaders(anime_csv='./data/2020/anime.csv', animelist_csv='./data/2020/animelist.csv')

# n_users = len(dataloader.dataset.X[:, 0].unique())
n_users = dataloader.dataset.X[:, 0].max()

# n_items = len(dataloader.dataset.X[:, 1].unique())
n_items = dataloader.dataset.X[:, 1].max()

In [7]:
class MatrixFactorization(nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()

        self.users_factors = nn.Embedding(n_users + 1, n_factors, dtype=torch.float32)
        self.items_factors = nn.Embedding(n_items + 1, n_factors, dtype=torch.float32)

        self.users_factors.weight.data.uniform_(0, 0.1)
        self.items_factors.weight.data.uniform_(0, 0.1)

    def forward(self, X):
        users, items = X[:, 0], X[:, 1]
        return (self.users_factors(users) * self.items_factors(items)).sum(1)
    
    def predict(self, X):
        return self.forward(X)

In [15]:
model = MatrixFactorization(n_users, n_items, n_factors=32).to(device)

In [16]:
loss_fn = MSELoss()
optimizer = Adam(model.parameters(), lr=0.0003)

In [17]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for (batch, (X, y)), it in zip(enumerate(dataloader), tqdm(range(len(dataloader)))):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 8000 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            print(it)

for it in tqdm(range(2)):
    print(it)
    train(dataloader, model, loss_fn, optimizer)

  0%|          | 0/2 [00:00<?, ?it/s]

0


  0%|          | 0/25535 [00:00<?, ?it/s]

loss: 36.891666  [  128/3268479]
0
loss: 14.410715  [1024128/3268479]
8000
loss: 11.982721  [2048128/3268479]
16000
loss: 9.964844  [3072128/3268479]
24000
1


  0%|          | 0/25535 [00:00<?, ?it/s]

loss: 8.765333  [  128/3268479]
0
loss: 9.340118  [1024128/3268479]
8000
loss: 8.716288  [2048128/3268479]
16000
loss: 8.376230  [3072128/3268479]
24000


In [18]:
with torch.no_grad():
    for X, y in dataloader:
        print(model(X.to(device)))
        break

tensor([ 4.8870e+00,  3.6491e+00,  1.0648e-01,  4.1295e-03,  2.5923e+00,
         3.4162e+00,  2.5316e+00,  7.3882e+00,  9.8570e+00,  6.8696e+00,
         8.4714e+00,  4.2914e+00,  1.1384e+00,  7.8132e-01,  5.6757e+00,
         1.5077e+00,  9.8350e-01,  5.9072e+00,  8.6180e+00,  1.8967e+00,
        -3.2273e-04,  1.2517e+00,  7.9205e-02,  6.6435e+00,  8.8029e+00,
         2.8759e+00,  5.4153e+00,  4.5798e+00,  5.5915e+00,  7.7585e+00,
         7.2467e+00,  3.2597e-01, -2.3073e-03,  4.8519e+00,  4.5392e+00,
         4.1101e+00,  7.7587e-01,  7.1731e+00,  5.3604e+00,  7.5192e+00,
         7.4152e+00,  8.4449e+00,  7.6797e+00,  6.2861e+00,  4.9725e+00,
         4.3446e+00,  7.6379e+00,  2.2655e+00,  3.8958e+00,  4.6676e+00,
         4.9968e+00,  7.6176e+00,  5.2193e+00,  3.1771e+00,  9.2011e+00,
         9.6923e-02,  4.5034e+00,  7.9497e+00,  5.5428e+00,  7.8051e+00,
         6.0845e-01,  8.4267e+00,  2.3527e+00,  9.1270e+00,  7.3240e+00,
         4.2550e+00,  7.9985e+00,  7.7096e+00,  7.4

In [23]:
trained_user_embeddings = model.users_factors.weight.cpu()
trained_item_embeddings = model.items_factors.weight.cpu()

In [24]:
torch.save(trained_user_embeddings, 'trained_user_embeddings_32.pt')
torch.save(trained_item_embeddings, 'trained_item_embeddings_32.pt')